<a href="https://colab.research.google.com/github/1ucky40nc3/stylegan3-super-recognizer/blob/main/generate_faces_with_stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# @title Check if a GPU is connected to the runtime
!nvidia-smi

Thu Oct 21 17:00:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    36W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
# @title Set up an environment (ONLY RUN ONCE)
!git clone https://github.com/NVlabs/stylegan3.git
!pip install ninja

%cd stylegan3/

downloaded_models = {}

fatal: destination path 'stylegan3' already exists and is not an empty directory.
/content/stylegan3


In [24]:
# @title Download a StyleGAN3 model
model = "stylegan3-t-ffhq-1024x1024.pkl" # @param ["stylegan3-t-ffhq-1024x1024.pkl", "tylegan3-t-ffhqu-1024x1024.pkl", "stylegan3-t-ffhqu-256x256.pkl", "stylegan3-r-ffhq-1024x1024.pkl", "stylegan3-r-ffhqu-1024x1024.pkl", "stylegan3-r-ffhqu-256x256.pkl", "stylegan3-t-metfaces-1024x1024.pkl", "stylegan3-t-metfacesu-1024x1024.pkl", "stylegan3-r-metfaces-1024x1024.pkl", "stylegan3-r-metfacesu-1024x1024.pkl", "stylegan3-t-afhqv2-512x512.pkl"]

if model not in downloaded_models:
    # Create the url to the resource.
    url = f"https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/{model}"
    print("[INFO] Downloading the model from NVIDIA.")

    # Download the pretrained model. 
    !wget $url

    # Mark the model as downloaded.
    downloaded_models[model] = url
else:
    print("[INFO] The model has already been downloaded.")

[INFO] The model has already been downloaded.


In [31]:
!python gen_images.py --outdir=out --trunc=1 --seeds=2 \
    --network=$model

Loading networks from "stylegan3-t-ffhq-1024x1024.pkl"...
Generating image for seed 2 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


In [32]:
!python gen_video.py --output=lerp.mp4 --trunc=1 --seeds=0-31 --grid=4x2 \
    --network=$model

Loading networks from "stylegan3-t-ffhq-1024x1024.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
100% 480/480 [08:48<00:00,  1.10s/it]
